# AI21 Agent with LangGraph and Tavily

This notebook demonstrates how to create a LangGraph agent with ChatAI21 model (using Jamba 1.5 that supports tools calling), and Tavily (search engine optimized for LLMs and RAG). 

To get Tavili API key, you need to sign up at [Tavily](https://tavily.com/).

In [1]:
!pip install langchain-ai21 tavily-python

  Using cached langchain_ai21-0.1.9-py3-none-any.whl.metadata (7.9 kB)
  Using cached tavily_python-0.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached ai21-2.15.0-py3-none-any.whl.metadata (22 kB)
  Using cached langchain_core-0.3.6-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached ai21_tokenizer-0.12.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached anyio-4.6.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached 

In [3]:
!pip install langchain_community 

  Using cached langchain_community-0.3.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached SQLAlchemy-2.0.35-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.10.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.6 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langchain-0.3.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pydantic_settings-2.5.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached aiohappyeyeballs-2.4.2-py3-none-any.whl.metadata (6.0 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.1.0-cp310-cp310-

In [5]:
!pip install langgraph

  Using cached langgraph-0.2.28-py3-none-any.whl.metadata (13 kB)
  Using cached langgraph_checkpoint-1.0.12-py3-none-any.whl.metadata (4.6 kB)
  Using cached msgpack-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
Using cached langgraph-0.2.28-py3-none-any.whl (107 kB)
Using cached langgraph_checkpoint-1.0.12-py3-none-any.whl (17 kB)
Using cached msgpack-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (378 kB)


In [6]:
import os

from langchain_ai21 import ChatAI21
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, StateGraph, START
import operator
from typing import Annotated, TypedDict, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]


os.environ["TAVILY_API_KEY"] = "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"
os.environ["AI21_API_KEY"] = "7rxSPvSTx0EYyrJBQHNyladQ274CNmRL" 


tools = [TavilySearchResults(max_results=5)]
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatAI21(model="jamba-1.5-large", temperature=0)
agent_runnable = create_tool_calling_agent(llm, tools, prompt=prompt)

tool_executor = ToolNode(tools)

def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

def execute_tools(data):
    outcomes = []
    for agent_action in data["agent_outcome"]:
        output = tool_executor.invoke(agent_action.message_log)
        outcomes.append((agent_action, str(output)))
    return {"intermediate_steps": outcomes}

def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

workflow.add_edge("action", "agent")

app = workflow.compile()

inputs = {
    "input": "Research and compile a comprehensive report on the current state of artificial intelligence "
             "in healthcare, highlighting the top companies and their innovative solutions.",
    "chat_history": []
}

for s in app.stream(inputs):
    print(list(s.values())[0])
    print("-----------------")

/home/igor/jambda/SocietyofScientists/.venv/lib/python3.10/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'agent_outcome': [ToolAgentAction(tool='tavily_search_results_json', tool_input={'query': 'current state of artificial intelligence in healthcare'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'current state of artificial intelligence in healthcare'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run-34fff41d-270c-4745-b1b5-495ff3e8d53e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current state of artificial intelligence in healthcare'}, 'id': 'db74d3a7-1dbc-445a-be66-fbc1b09a4a1d', 'type': 'tool_call'}])], tool_call_id='db74d3a7-1dbc-445a-be66-fbc1b09a4a1d')]}
-----------------
